<h1><center>Laboratorio 5: La desperación de Mr. Cheems 🐼</center></h1>

<center><strong>MDS7202: Laboratorio de Programación Científica para Ciencia de Datos - Primavera 2024</strong></center>

### Cuerpo Docente:

- Profesores: Ignacio Meza, Sebastián Tinoco
- Auxiliar: Eduardo Moya
- Ayudantes: Nicolás Ojeda, Melanie Peña, Valentina Rojas

### Equipo: SUPER IMPORTANTE - notebooks sin nombre no serán revisados

- Nombre de alumno 1: Cristobal Ramos
- Nombre de alumno 2: Catalina Lizana


### **Link de repositorio de GitHub:** [Repositorio](https://github.com/CatalinaLizanaG/Laboratorio-de-Programaci-n.git)

## Temas a tratar
- Aplicar Pandas para obtener características de un DataFrame.
- Aplicar Pipelines y Column Transformers

## Reglas:

- **Grupos de 2 personas**
- Fecha de entrega: 6 días de plazo con descuento de 1 punto por día. Entregas Jueves a las 23:59.
- Instrucciones del lab el viernes a las 16:15 en formato online. Asistencia no es obligatoria.
- <u>Prohibidas las copias</u>. Cualquier intento de copia será debidamente penalizado con el reglamento de la escuela.
- Tienen que subir el laboratorio a u-cursos y a su repositorio de github. Labs que no estén en u-cursos no serán revisados. Recuerden que el repositorio también tiene nota.
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Pueden usar cualquier material del curso que estimen conveniente.

### Objetivos principales del laboratorio
- Comprender cómo aplicar pipelines de Scikit-Learn para generar procesos más limpios en Feature Engineering.

El laboratorio deberá ser desarrollado sin el uso indiscriminado de iteradores nativos de python (aka "for", "while"). La idea es que aprendan a exprimir al máximo las funciones optimizadas que nos entrega `numpy`, las cuales vale mencionar, son bastante más eficientes que los iteradores nativos sobre arreglos (*o tensores*).

## Descripción del laboratorio.

### Importamos librerias utiles 😸

In [ ]:
# Libreria Core del lab.
import numpy as np
import pandas as pd
import datetime
from IPython.display import HTML

# Libreria para plotear (En colab esta desactualizado plotly)
#!pip install --upgrade plotly
import plotly.express as px
import plotly.graph_objects as go

# Librerias utiles
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import FunctionTransformer

In [ ]:
# Si usted está utilizando Colabolatory le puede ser útil este código para cargar los archivos.
#try:
 #   from google.colab import drive
  #  drive.mount("/content/drive")
  #  path = 'Dirección donde tiene los archivos en el Drive'
#except:
 #   print('Ignorando conexión drive-colab')

# Feature engineering en datos de retail 🛍️

### 0. Cargar Dataset

<p align="center">
  <img width=300 src="https://s1.eestatic.com/2018/04/14/social/la_jungla_-_social_299733421_73842361_854x640.jpg">
</p>

Mr. Cheems, gerente de una cotizada tienda de retail en Europa, les solicita si pueden analizar los datos de algunas de sus tiendas. En una reunión, Mr Cheems le comenta que la calidad de sus datos no es muy buena, por lo que le solicita a usted que limpie su base de datos y cree nuevos atributos relevantes para el negocio.

Por ello, el área de ventas les entrega archivo llamado `online_retail_data.pickle` el cual usted decide cargar a continuación.

In [ ]:
df_retail = pd.read_pickle("online_retail_data.pickle")
df_retail.head(5)

### 1. Función para explorar características [5 puntos]

<p align="center">
  <img width=300 src="https://editor.analyticsvidhya.com/uploads/47389meme.png">
</p>




Tras inspeccionar brevemente los datos proporcionados, usted decide crear una función que realice lo siguiente:
- Plotee un histograma para las variables precios y cantidad. [3 puntos]
- Imprima un conteo de datos nulos por variable [2 puntos]

NOTA: Para generar los gráficos es **OBLIGATORIO** el uso de plotly.

**Respuesta:**

In [ ]:
def explore_data(dataframe_in, title_descr=''):
    #Plotee un histograma para las variables precios y cantidad.
    hist_price = px.histogram(dataframe_in, x="Price", nbins=10)
    hist_price.update_layout(title_text="Histograma precios "+title_descr)
    hist_price.show()

    hist_quant = px.histogram(dataframe_in, x="Quantity", nbins=10)
    hist_quant.update_layout(title_text="Histograma cantidad "+title_descr)
    hist_quant.show()
    
    #Imprima un conteo de datos nulos por variable
    nulos = dataframe_in.isna().sum(axis=0)
    print('Conteo de datos nulos por variable')
    print(nulos)
  
#pass

In [ ]:
explore_data(df_retail, title_descr="df_retail")

### 2. Eliminando outliers [10 puntos]

<p align="center">
  <img width=300 src="https://media.licdn.com/dms/image/C5612AQGdXKCka7HumA/article-cover_image-shrink_600_2000/0/1520056407281?e=2147483647&v=beta&t=VZcfjjzjK4LxXdZkSu1KisWC0Ry8bk4tPCn3R8aYdNM">
</p>




#### 2.1 Creando la clase IQR [5 puntos]

Entre las falencias de los datos, Mr. Cheems le comenta que a veces los operadores no ingresan el precio correcto de los productos. Mr. Cheems le comenta que se dio cuenta de este fenómeno porque hay productos con precios exagerádamente altos o bajos. Por lo cual usted decide eliminar outliers del dataframe a traves del rango intercuartil el cual cuenta con los siguientes pasos:

1. Calcular el primer cuartil $Q1$ y el tercer cuartil $Q3$. Hint: utilice el método `quantile()`

2. Calcular el rango intercuartil (RIC): $RIC = Q3 - Q1$

3. Calcular los límites para identificar outliers:
 - Límite inferior: $~~Q1 - \lambda \cdot RIC$
 - Límite superior: $~~Q3 + \lambda \cdot RIC$

4. Eliminar outliers: Los outliers son los datos que están por debajo del límite inferior o por encima del límite superior.


Para realizar dicha tarea, usted decide crear una clase llamada `IQR()` utilizando `BaseEstimator` y `TransformerMixin` para realizar una transformación de cada una de las columnas numéricas del DataFrame utilizando `ColumnTransformer()` más tarde. Considere que lambda debe ser $\lambda$ un parámetro a definir por el usuario.

Hint: tome como referencia el siguiente [enlace](https://sklearn-template.readthedocs.io/en/latest/user_guide.html#transformer).
>**Nota: No modificar el método set_output de la clase IQR**

**Respuesta:**

In [ ]:
class IQR(BaseEstimator, TransformerMixin):

  def __init__(self, l):
    self.l = l

  def fit(self, X):
    return self

  def transform(self, X):
    q1 = X.quantile(0.25)
    q3 = X.quantile(0.75)
    RIC = q3-q1
    li = q1-self.l*RIC
    ls = q3+self.l*RIC
    mask = (X < li)|(X>ls)
    return X[~mask] 

    #pass

  def set_output(self,transform='default'):
    #No modificar esta función
    return self

#### 2.2 Creación del Pipeline [5 puntos]

Para comenzar introduciéndose en el uso de pipeline, usted decide definir un pipeline con el Transformer previamente definido. Además, usted decide visualizar cómo cambia la distribución de las variables Precio y Cantidad antes y despues de aplicar IQR. Para ello, usted aplica los siguientes pasos:

- Definir un pipeline llamado `numeric_transformations` para las variables precio y cantidad con la transformación IQR. [1 punto]
- Defina un column transformer que aplique `numeric_transformations` para las variables numéricas y `passthrough` para las variables categóricas. Adicionalmente, fije el parámetro `verbose_feature_names_out` en `False`. Ver hint al final [1 puntos]
- Defina el dataframe `df_iqr` aplicado el column transformer a los datos proporcionados por Mr. Cheems considerando un valor de $\lambda$ que tenga un desempeño aceptable para ambas variables. [1 punto]
- Usar `explore_data` en `df_retail` y en `df_iqr`.  [1 punto]
- Reportar los cambios observados en la distribución de las variables. ¿Qué sucede al aumentar el valor de lambda? [1 punto]


HINT: El transformador `passthrough` está predefinido y es una opción que puedes usar para las columnas que no deseas transformar. Al especificar 'passthrough' para una parte de tu ColumnTransformer, las columnas correspondientes pasarán a través del ColumnTransformer sin ninguna modificación. El siguiente [enlace](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html) le puede ser útil.
>**Nota: Mantenga el método set_output del column transformer con la transformación `pandas` para obtener un dataframe una vez aplicado el column transformer.**

**Respuesta:**

Apóyese de la siguiente estructura para su respuesta:

In [ ]:
# Definicion las variables que pasarán por cada pipeline
numerical_columns = df_retail.select_dtypes('float').columns
categorical_columns = ['Invoice', 'StockCode', 'Description', 'InvoiceDate', 'Customer ID', 'Country']

# Definicion del pipeline
numeric_transformations = Pipeline([('IQR',IQR(5))])

# ColumnTransformer
column_transformer = ColumnTransformer([('numerical', numeric_transformations, numerical_columns),
                                        ('categorical', 'passthrough', categorical_columns)
                                        ],
                                        verbose_feature_names_out=False)

column_transformer.set_output(transform='pandas')

# Aplicamos ColumnTransformer a los datos
df_iqr = column_transformer.fit_transform(df_retail)


In [ ]:
# Gráficos
explore_data(df_retail, title_descr="df_retail")

In [ ]:
explore_data(df_iqr, title_descr="df_iqr")

*Reporte los cambios observados aquí.*

 ¿Qué sucede al aumentar el valor de lambda?

 A mayor Lambda, mayor es el rango de valores que quedan (menos outliers) \
 λ = 1:  Price $\in [-0.5,6.49]$, 
         Quantity $\in [0,24]$
       
 λ = 5: Price $\in [-1,16.99]$,
       Quantity $\in [0,69]$

 λ = 8: Price $\in [-2.5,22.49]$,
       Quantity $\in [0,99]$
       
 λ = 10: Price $\in [-2.5,32.49]$,
       Quantity $\in [0,119]$
    


#### 3. Agregando un imputer al pipeline [10 puntos]



<p align="center">
  <img width=300 src="https://media.makeameme.org/created/hmm-there-is.jpg">
</p>

Para continuar con la limpieza del dataframe usted decide imputar los datos nulos de las variables numéricas, para lo cual decide realizar las siguientes tareas:

1. Crear un pipeline para variables categóricas llamado `categoric_transformations` con un paso llamado `mode_imputer`, en el cual se imputen los datos faltantes por la categoría más frecuente.
2. Agregar al pipeline `numeric_transformations` un paso llamado `mean_imputer`, en el cual se imputen los datos por la media usando [SimpleImputer](https://scikit-learn.org/stable/modules/generated/sklearn.impute.SimpleImputer.html) [1 punto]
3. Crear y aplicar un `ColumnTransformer` actualizado con los pipelines `categoric_transformations` y `numeric_transformations` a `df_retail`, creando un dataframe llamado `df_mean_imputer`. [1 punto]
4. Comparar los resultados de `explore_data` en `df_mean_imputer` y `df_iqr`. ¿Qué diferencias observa en la distribución de los datos? [2 puntos]
5. Cambiar el imputer de `numeric_transformations` por [KNNImputer](https://scikit-learn.org/stable/modules/generated/sklearn.impute.KNNImputer.html) y definir un nuevo dataframe llamado `df_knn_imputer`, aplicando el nuevo ColumnTransformer a `df_retail`. En caso de los tiempos de ejecución sean altos puede probar a reducir el parámetro `n_neighbors`. [1 punto]
6. Comparar los resultados de `explore_data` en `df_knn_imputer` y `df_iqr`. ¿Qué diferencias observa en la distribución de los datos? [2 puntos]
7. Comparar los resultados de `explore_data` en `df_knn_imputer` y `df_mean_imputer`. ¿Cuál método de imputación es mejor? Deje el método escogido en el ColumnTransformer. [2 puntos]

>**Nota: Fije el parámetro verbose_feature_names_out en `False` y utilice el método set_output con transformación `pandas` en cada ColumnTransformer para obtener como salida un dataframe.**

**Respuesta:**

In [ ]:
from sklearn.impute import SimpleImputer
#1.Crear un pipeline para variables categóricas llamado categoric_transformations con un paso llamado mode_imputer, 
#en el cual se imputen los datos faltantes por la categoría más frecuente.
categoric_transformations = Pipeline([('mode_imputer',SimpleImputer(strategy='most_frequent') )])

#2.Agregar al pipeline numeric_transformations un paso llamado mean_imputer, 
#en el cual se imputen los datos por la media usando SimpleImputer 
numeric_transformations = Pipeline([('IQR',IQR(5)), ('mean_imputer', SimpleImputer(strategy="median"))])

#3.Crear y aplicar un ColumnTransformer actualizado con los pipelines categoric_transformations y numeric_transformations a df_retail, 
#creando un dataframe llamado df_mean_imputer. 
column_transformer = ColumnTransformer([('numerical', numeric_transformations, numerical_columns),
                                        ('categorical',categoric_transformations , categorical_columns)],
                                        verbose_feature_names_out=False)

column_transformer.set_output(transform='pandas')
df_mean_imputer = column_transformer.fit_transform(df_retail)

In [ ]:
#4.Comparar los resultados de explore_data en df_mean_imputer y df_iqr.
explore_data(df_mean_imputer, title_descr="df_mean_imputer")

In [ ]:
#5.Cambiar el imputer de numeric_transformations por KNNImputer y definir un nuevo dataframe llamado df_knn_imputer, 
#aplicando el nuevo ColumnTransformer a df_retail.

from sklearn.impute import KNNImputer
KNNimputer = KNNImputer(n_neighbors=1)

numeric_transformations = Pipeline([('IQR',IQR(5)), ('mean_imputer', KNNimputer)])

column_transformer = ColumnTransformer([('numerical', numeric_transformations, numerical_columns),
                                        ('categorical',categoric_transformations , categorical_columns)],
                                        verbose_feature_names_out=False)

column_transformer.set_output(transform='pandas')
df_knn_imputer = column_transformer.fit_transform(df_retail)


In [ ]:
#6.Comparar los resultados de explore_data en df_knn_imputer y df_iqr.
#7.Comparar los resultados de explore_data en df_knn_imputer y df_mean_imputer. 
explore_data(df_knn_imputer, title_descr="df_knn_imputer")

*Escriba su respuesta aquí*

4. Comparar los resultados de explore_data en df_mean_imputer y df_iqr:
A simple vista se evidencia que los histogramas generados presentan distribuciones semjantes, sin embargo, los resultados en df_mean_imputer presentan más datos en el rango (1-2.99) en price y más datos en el rango (0-10) en cantidad, en comparación con los datos df_iqr.

6. Comparar los resultados de explore_data en df_knn_imputer y df_iqr ¿Qué diferencias observa en la distribución de los datos?
No se evidencian diferencias significativas en los datos de los histogramas df_knn_imputer y df_iqr, sin embargo al observar mejor notamos que df_iqr presenta más valores en los rangos (1-2.99) y (0-10) de price y cantidad respectivamente.

8. Comparar los resultados de explore_data en df_knn_imputer y df_mean_imputer ¿Cuál método de imputación es mejor?
Los histogramas generados con df_mean_imputer presentan más datos en los rangos de media en comparación con los datos df_knn_imputer.
El mejor método es df_knn_imputer, ya que, al identificar los valores basados en los datos de los 'vecinos cercanos' es mejor la asignación de valores a los datos faltantes, sin embargo, el tiempo de ejecucióne es considerablemente mayor que con df_mean_imputer




### 4. Creación de nuevas features [20 puntos]

<p align="center">
  <img width=250 src="https://miro.medium.com/max/1000/1*JtTWgAcfVTWV8OTjT47Atg.jpeg">
</p>


#### 4.1 Definicion de LRMFP [10 puntos]
(2 puntos por cada custom feature)

Dado que Mr. Lepin está interesado en obtener nuevos atributos relevantes para su negocio, su equipo de expertos sugiere la construcción de variables **LRMFP**, las que se construyen en base a las siguientes definiciones:

- **Length (L)**: Intervalo de tiempo, en días, entre la primera y la última visita del cliente. Mientras mas grande sea el valor, mas fiel es el cliente.

- **Recency (R)**: Indica hace cuanto tiempo el cliente realizo su ultima compra. Notar que para este caso, mientras mas grande es el valor, menos interes posee el usuario para repetir una compra en uno de los locales. **Considere "hoy" como la fecha mas reciente del dataset**.

- **Monetary (M)**: El término "monetario" se refiere a la cantidad media de dinero gastada por cada visita del cliente durante el período de observación y refleja la contribución del cliente a los ingresos de la empresa.

- **Frequency (F)**: Se refiere al número total de visitas del cliente durante el periodo de observación. Cuanto mayor sea la frecuencia, mayor será la fidelidad del cliente.

- **Periodicity (P)**: Representa si los clientes visitan las tiendas con regularidad.

$$Periodicity(n)=std(IVT_1, ..., IVT_n)$$

Donde $IVT$ denota el tiempo entre visitas y n representa el número de valores de tiempo entre visitas de un cliente.


$$IVT_i=date\_diff(t_{i+1},t)$$

En base a las definiciones señaladas, diseñe una función que permita obtener las características **LRMFP** recibiendo un DataFrame como entrada. Para esto, no estará permitido el uso de iteradores, utilice todas las herramientas que les ofrece `pandas` para realizar esto.

Una referencia que le puede ser útil es el [documento original](https://www.researchgate.net/publication/315979555_LRFMP_model_for_customer_segmentation_in_the_grocery_retail_industry_a_case_study) en donde se propone este método.

**<u>Formato</u> del Resultado Esperado:**

| Customer ID | Length | Recency | Frequency | Monetary | Periodicity |
|------------:|-------:|--------:|----------:|---------:|------------:|
|   12346.0   |    294 |      67 |        46 |   -64.68 |        37.0 |
|   12347.0   |     37 |       3 |        71 |  1323.32 |         0.0 |
|   12349.0   |    327 |      43 |       107 |  2646.99 |        78.0 |
|   12352.0   |     16 |      11 |        18 |   343.80 |         0.0 |
|   12356.0   |     44 |      16 |        84 |  3562.25 |        12.0 |

**Respuesta:**

In [ ]:
from datetime import date

def custom_features(dataframe_in):
    LRMFP = pd.DataFrame()

    times = dataframe_in.groupby("Customer ID", observed=False).agg({"InvoiceDate": ["max", "min"]})
    today=date.today()

    # Length (L): Intervalo de tiempo, en días, entre la primera y la última visita del cliente. Mientras mas grande sea el valor, mas fiel es el cliente.
    LRMFP['Length'] = (times["InvoiceDate","max"]-times["InvoiceDate","min"]).dt.days

    # Recency (R): Indica hace cuanto tiempo el cliente realizo su ultima compra. Mientras mas grande es el valor, menos interes posee el usuario. Considere "hoy" como la fecha mas reciente del dataset.
    LRMFP['Recency'] = (pd.to_datetime(today)-times["InvoiceDate","max"]).dt.days

    # Monetary (M): Se refiere a la cantidad media de dinero gastada por cada visita del cliente durante el período de observación y refleja la contribución del cliente a los ingresos de la empresa.
    dataframe_in['money'] = dataframe_in['Quantity']*dataframe_in['Price']
    LRMFP['Monetary']= dataframe_in.groupby("Customer ID", observed=False).agg({"money": 'sum'})

    # Frequency (F): Se refiere al número total de visitas del cliente durante el periodo de observación. Cuanto mayor sea la frecuencia, mayor será la fidelidad del cliente.
    LRMFP['Frequency']= dataframe_in.groupby("Customer ID", observed=False)['Invoice'].count()
    
    # Periodicity (P): Representa si los clientes visitan las tiendas con regularidad. Periodicity(n)=std(IVT_1, ..., IVT_n), donde IVT denota el tiempo entre visitas y n representa el número de valores de tiempo entre visitas de un cliente.
    dataframe_in = dataframe_in.sort_values(by=['Customer ID','InvoiceDate'])
    dataframe_in['dt']=dataframe_in['InvoiceDate'].diff()
    LRMFP['Periodicity'] = dataframe_in.groupby("Customer ID", observed=False).agg({"dt": 'std'})
    LRMFP['Periodicity'] = LRMFP['Periodicity'].dt.days
    
    return LRMFP

In [ ]:
custom_features(df_retail).head(10)

#### 4.2 Agregando las custom features [10 puntos]

Ahora, usted decide agregar al pipeline las nuevas variables creadas, para lo cual realiza las siguientes tareas:

1. Cree un nuevo pipeline llamado `retail_pipeline` que encapsule el ColumnTransformer y calcule las LRMFP. El primer paso del pipeline llámelo  `col_tranformer` y el segundo paso llámelo `custom_features`, incorpora las nuevas variables al dataframe. Hint: les puede ser útil investigar [este](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.FunctionTransformer.html) método. [1 punto]
2. Aplicar el pipeline actualizado a los datos proporcionados por Mr. Cheems, creando un nuevo dataframe llamado `df_custom`. [1 punto]
3. Explorar la distribución de las nuevas variables con `explore_data` y comentar brevemente (2-3 líneas) características de cada custom feature. [5 puntos]
5. Entregar un insight para el negocio en base a las nuevas variables. [3 puntos]

>Nota: Recuerde fijar el parámetro `verbose_feature_names_out` en `False` e incorporar el método `set_output` para obtener una salida en formato dataframe del ColumnTransformer.

**Respuesta**

In [ ]:
#1.Cree un nuevo pipeline llamado retail_pipeline que encapsule el ColumnTransformer y calcule las LRMFP. 
#El primer paso del pipeline llámelo col_tranformer y el segundo paso llámelo custom_features, incorpora las nuevas variables al dataframe.
from sklearn.preprocessing import FunctionTransformer

transformer = FunctionTransformer(custom_features)

retail_pipeline = Pipeline([('col_tranformer', column_transformer), ('custom_features', transformer)]) #Usamos column_transformer ya definido
retail_pipeline.set_output(transform='pandas')

# 2. Aplicar el pipeline actualizado a los datos proporcionados por Mr. Cheems, creando un nuevo dataframe llamado df_custom
df_custom = retail_pipeline.fit_transform(df_retail)
df_custom

In [ ]:
def explore_data_LRMFP(dataframe_in):
    #Plotee un histograma para las variables LRMFP
    hist_length = px.histogram(dataframe_in, x="Length", nbins=10)
    hist_length.update_layout(title_text="Histograma Length")
    hist_length.show()

    hist_recency = px.histogram(dataframe_in, x="Recency", nbins=10)
    hist_recency.update_layout(title_text="Histograma Recency")
    hist_recency.show()

    hist_monetary = px.histogram(dataframe_in, x="Monetary", nbins=10)
    hist_monetary.update_layout(title_text="Histograma  Monetary")
    hist_monetary.show()

    hist_frequency = px.histogram(dataframe_in, x="Frequency", nbins=10)
    hist_frequency.update_layout(title_text="Histograma Frequency")
    hist_frequency.show()

    hist_periodicity = px.histogram(dataframe_in, x="Periodicity", nbins=10)
    hist_periodicity.update_layout(title_text="Histograma Periodicity")
    hist_periodicity.show()
    
    #Imprima un conteo de datos nulos por variable
    nulos = dataframe_in.isna().sum(axis=0)
    print('Conteo de datos nulos por variable')
    print(nulos)

In [ ]:
#3.Explorar la distribución de las nuevas variables con explore_data y comentar brevemente características de cada custom feature. 
explore_data_LRMFP(df_custom)

4. Entregar un insight para el negocio en base a las nuevas variables:

   
L: Notamos que muchos de los clientes el intervalo de tiempo entre la primera y la última visita es entre 0 y 50 dias, sin embargo, más del 50% de los clientes tienen un length de más de 200 dias (son fieles)

R: el cliente más reciente fue hace 5000 dias, y la mayoria lleva entre 5000 y 5200 dias sin ir, estos son los cliente más fieles en comparación a los otros

M: En general, los clientes gastan en promedio menos de 10k en cada visita.

F: La gran mayoria de los cliente fue entre 0 y 1000 dias. Hay 15 clientes que fueron más de 1000 días, estos son los más fieles.

P: En general se puede decir que los clientes visitan regularmente la tienda.


### 5. MinMax Scaler [10 puntos]

<p align="center">
  <img width=300 src="https://i.imgflip.com/1fsprn.jpg">
</p>


#### 5.1 Definición del Column Transformer [5 puntos]

Construya una clase llamada `MinMax()` para realizar una transformación de cada una de las columnas de un DataFrame utilizando `ColumnTransformer()`. Recuerde  usar `BaseEstimator` y `TransformerMixin`.


 Para esto considere que Min-Max escaler queda dada por la ecuación:

$$MinMax = \dfrac{x-min(x)}{max(x) - min(x)}$$


Consulte el siguiente [link](https://sklearn-template.readthedocs.io/en/latest/user_guide.html#transformer) si tiene dudas sobre la creación de custom transformers.

**Respuesta:**

In [ ]:
class MinMax(BaseEstimator, TransformerMixin):

    def fit(self,X):
        return self
        #pass

    def transform(self,X):
        num= X - X.min()
        den= X.max()-X.min()
        return num/den
        #pass

    def set_output(self,transform='default'):
        #No modificar este método
        return self



#### 5.2 Incorporando MinMax al pipeline [5 puntos]

Ahora, usted decide agregar el escalamiento al pipeline, para lo que decide seguir los siguientes pasos:

- Agregar el paso `minmax` al pipeline `numeric_transformations`, haciendo uso de la clase creada. [1 punto]
- Defina el dataframe `df_minmax` aplicando el ColumnTransformer actualizado a los datos proporcionados por Mr. Cheems. [1 punto]
- Usar `explore_data` en `df_retail` y en `df_minmax`. [1 punto]
- Reportar los cambios observados en la distribución de las variables.  [2 puntos]

>Nota: Recuerde fijar el parámetro `verbose_feature_names_out` en `False` e incorporar el método `set_output` para obtener una salida en formato dataframe del ColumnTransformer.

**Respuesta:**

In [ ]:
#1.Agregar el paso minmax al pipeline numeric_transformations, haciendo uso de la clase creada. 
numeric_transformations = Pipeline([('IQR',IQR(5)), ('mean_imputer', SimpleImputer(strategy="median")),('minmax',MinMax())])

#2.Defina el dataframe df_minmax aplicando el ColumnTransformer actualizado a los datos proporcionados por Mr. Cheems. 
column_transformer = ColumnTransformer([('numerical', numeric_transformations, numerical_columns),
                                        ('categorical',categoric_transformations , categorical_columns)],
                                        verbose_feature_names_out=False)

column_transformer.set_output(transform='pandas')
df_minmax = column_transformer.fit_transform(df_retail)
df_minmax

In [ ]:
#3.Usar explore_data en df_retail y en df_minmax
explore_data(df_retail, title_descr="df_retail")

In [ ]:
explore_data(df_minmax, title_descr="df_minmax")

4. Reportar los cambios observados en la distribución de las variables. \
Observando los histogramas generados con df_minmax vemos que las distribuciones de las variables toman valores en el intervalo $[-0.1,1.1)$, a diferencia del df original, en donde las variables "Price" toma valores entre $0$ y $10k$, y "Quantity" entre $0$ y $20k$.
Mientras que en el df original los valores de las variables se concentran en único rango de cada histograma, los valores de las variables de df_minmax se distribuyen en distintos rangos del intervalo total, concentrando la mayor cantidad de valores de la variable "price" en el rango $[0.1 - 0.29)$, y la mayor cantidad de valores de "Quantity" en el rango $[-0.1 - 0.099)$. Así, vemos que la distribución generada por df_minmax aporta más información para el análisis de los datos. 

### 6. Pregunta teórica [5 puntos]

<p align="center">
  <img width=300 src="https://file.coinexstatic.com/2023-09-19/166BAC031F222E5910954E7D7D0BC844.png">
</p>

Finalmente, explíquele a Mr. Cheems porqué es útil la creación de pipelines al momento de hacer Feature Engineering en Machine Learning.

**Respuesta:**

El uso de **`pipelines`** nos ayuda a **simplificar el proceso de aprendizaje automático**. Esto pues, en un pipeline es posible combinar muchos de los pasos necesarios para generar el aprendizaje de la máquina en una única entidad, **simplificando el flujo de trabajo y mejorando la legibilidad** del mismo. Así, proporcionando de forma clara una organización del código, los pipelines reducen la posibilidad de cometer errores en el proceso (al simplificar y reducir la manipulación de los datos) y aportan una mayor facilidad en (entre otros procesos) el **ajuste de hiperparámetros** de varias etapas del proceso de aprendizaje, dando más oportunidades de **mejora en el rendimiento del modelo**. 

Por otro lado, se obtiene un **procesamiento de datos más consistente** al asegurar que los pasos realizados en el preprocesamiento de los datos de entrenamiento se puedan replicar en los datos de prueba, disminuyendo el riesgo de sobreajuste y facilitando una mejor comparación con otros modelos en cuanto al rendimiento del modelo obtenido. 
complejos.

# Conclusión
Eso ha sido todo para el lab de hoy, recuerden que el laboratorio tiene un plazo de entrega de una semana. Cualquier duda del laboratorio, no duden en contactarnos por el foro de U-cursos o por correo.

![Gracias Totales!](https://i.pinimg.com/originals/65/ae/27/65ae270df87c3c4adcea997e48f60852.gif "bruno")


<br>
<center>
<img src="https://i.kym-cdn.com/photos/images/original/001/194/195/b18.png" width=100 height=50 />
</center>
<br>

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=87110296-876e-426f-b91d-aaf681223468' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>